In [1]:
import torch
import torch.nn.functional as F
from  reorient_nii import reorient_1
import os
import numpy as np
import matplotlib.pyplot as plt

/home1/ruitongs/.conda/envs/UDA/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import nibabel as nib
import nilearn.plotting as nlplt
from nibabel.testing import data_path

/home1/ruitongs/.conda/envs/UDA/lib/python3.7/site-packages/nilearn/__init__.py:67: FutureWarning: Python 3.7 support is deprecated and will be removed in release 0.12 of Nilearn. Consider switching to Python 3.9 or 3.10.
  _python_deprecation_warnings()


In [3]:
folder_path1 = 'ct_test1/'
folder_path2 = 'ct_test2/'
folder_path3 = 'ct_test3/'
folder_path4 = 'ct_test4/'
save_folder_path = 'ct_npy_unlabel/'

In [4]:
def reset_label_value(label):
    
    label[label == 820] = 4 # AA  # small isolated
    label[label == 500] = 3 # LV  # center 
    label[label == 420] = 2 # LA  # long tail 
    label[label == 205] = 1 # Myo # blue semi-cicle close to red   

    label[label == 550] = 0
    label[label == 600] = 0
    label[label == 850] = 0
    
    return label

In [5]:
def corp_base_on_min_max_label(image , label):

    heart_indices = np.where(label != 0)
    min_coords = np.min(heart_indices, axis=1)
    max_coords = np.max(heart_indices, axis=1)
    
    cropped_image = image[min_coords[0]:max_coords[0], min_coords[1]:max_coords[1], min_coords[2]:max_coords[2]]
    cropped_label = label[min_coords[0]:max_coords[0], min_coords[1]:max_coords[1], min_coords[2]:max_coords[2]]
    
    return np.array(cropped_image), np.array(cropped_label)

In [6]:
def resample_label(nii, label):
    
    label_header = nii.header
    
    target_shape = [int(label.shape[0] * label_header['pixdim'][1]), int(label.shape[1] * label_header['pixdim'][2]),\
                int(label.shape[2] * label_header['pixdim'][3])]
    
    
    label = torch.from_numpy( label.copy() ).unsqueeze(0).unsqueeze(0) # torch.Size([1, 1, 512, 512, 84])
    label = F.interpolate( label, target_shape, mode = "nearest").numpy()[0,0] # (342, 342, 63)
    
    return label

In [7]:
def resample_img(nii, image):
    
    image_header = nii.header
    
    target_shape = [int(image.shape[0] * image_header['pixdim'][1]), int(image.shape[1] * image_header['pixdim'][2]),\
                int(image.shape[2] * image_header['pixdim'][3])]
    
    
    image = torch.from_numpy( image.copy() ).unsqueeze(0).unsqueeze(0) # torch.Size([1, 1, 512, 512, 84])
    image = F.interpolate( image, target_shape, mode = "trilinear").numpy()[0,0] # (342, 342, 63)
    
    return image

In [8]:
for idx in range(2021, 2031):
    print(idx)

2021
2022
2023
2024
2025
2026
2027
2028
2029
2030


In [9]:
for idx in range(2001, 2011):
    
    '''image'''
    image_filepath = os.path.join(folder_path1 + f'ct_test_{idx}_image.nii.gz')
    img_0 = nib.load(image_filepath)
    img = reorient_1(img_0)
    image_data = img.get_fdata(dtype=np.float32)
    image_data = resample_img(img_0, image_data)
    
    # Clip the top 2% of the intensity histogram
    percentile_98 = np.percentile(image_data.ravel(), 98)
    image_data = np.clip(image_data, a_min=None, a_max=percentile_98)
    
    # Subtract the mean and divide by the standard deviation
    mean_val = np.mean(image_data)
    std_val = np.std(image_data)
    image_data = (image_data - mean_val) / std_val
    
    print(image_data.shape)
    np.save(f"{save_folder_path}ct_test_{idx}_image.npy", image_data)

(231, 231, 140)
(163, 163, 125)
(249, 249, 172)
(192, 192, 128)
(236, 236, 169)
(236, 236, 169)
(291, 291, 180)
(265, 265, 163)
(288, 288, 187)
(258, 258, 171)


In [11]:
for idx in range(2011, 2021):
    
    '''image'''
    image_filepath = os.path.join(folder_path2 + f'ct_test_{idx}_image.nii.gz')
    img_0 = nib.load(image_filepath)
    img = reorient_1(img_0)
    image_data = img.get_fdata(dtype=np.float32)
    image_data = resample_img(img_0, image_data)
    
    # Clip the top 2% of the intensity histogram
    percentile_98 = np.percentile(image_data.ravel(), 98)
    image_data = np.clip(image_data, a_min=None, a_max=percentile_98)
    
    # Subtract the mean and divide by the standard deviation
    mean_val = np.mean(image_data)
    std_val = np.std(image_data)
    image_data = (image_data - mean_val) / std_val
    
    print(image_data.shape)
    np.save(f"{save_folder_path}ct_test_{idx}_image.npy", image_data)

(179, 179, 134)
(215, 215, 160)
(249, 249, 179)
(231, 231, 140)
(291, 291, 180)
(223, 223, 151)
(249, 249, 155)
(224, 224, 131)
(249, 249, 185)
(155, 155, 134)


In [12]:
for idx in range(2021, 2031):
    
    '''image'''
    image_filepath = os.path.join(folder_path3 + f'ct_test_{idx}_image.nii.gz')
    img_0 = nib.load(image_filepath)
    img = reorient_1(img_0)
    image_data = img.get_fdata(dtype=np.float32)
    image_data = resample_img(img_0, image_data)
    
    # Clip the top 2% of the intensity histogram
    percentile_98 = np.percentile(image_data.ravel(), 98)
    image_data = np.clip(image_data, a_min=None, a_max=percentile_98)
    
    # Subtract the mean and divide by the standard deviation
    mean_val = np.mean(image_data)
    std_val = np.std(image_data)
    image_data = (image_data - mean_val) / std_val
    
    print(image_data.shape)
    np.save(f"{save_folder_path}ct_test_{idx}_image.npy", image_data)

(186, 186, 116)
(249, 249, 149)
(229, 229, 143)
(249, 249, 172)
(192, 192, 128)
(216, 216, 154)
(199, 199, 149)
(144, 144, 161)
(179, 179, 134)
(249, 249, 183)


In [13]:
for idx in range(2031, 2041):
    
    '''image'''
    image_filepath = os.path.join(folder_path4 + f'ct_test_{idx}_image.nii.gz')
    img_0 = nib.load(image_filepath)
    img = reorient_1(img_0)
    image_data = img.get_fdata(dtype=np.float32)
    image_data = resample_img(img_0, image_data)
    
    # Clip the top 2% of the intensity histogram
    percentile_98 = np.percentile(image_data.ravel(), 98)
    image_data = np.clip(image_data, a_min=None, a_max=percentile_98)
    
    # Subtract the mean and divide by the standard deviation
    mean_val = np.mean(image_data)
    std_val = np.std(image_data)
    image_data = (image_data - mean_val) / std_val
    
    print(image_data.shape)
    np.save(f"{save_folder_path}ct_test_{idx}_image.npy", image_data)

(249, 249, 179)
(158, 158, 110)
(224, 224, 138)
(216, 216, 154)
(215, 215, 160)
(157, 157, 149)
(229, 229, 143)
(265, 265, 163)
(158, 158, 110)
(300, 300, 176)
